<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_MobileNetV1/test_sample_MobileNetV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

In [ ]:
! pip install datasets
! pip install wget

In [1]:
# ! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/

fatal: destination path 'PyTorch-Architectures' already exists and is not an empty directory.
/content/PyTorch-Architectures


In [2]:
from tqdm.auto import tqdm
import torch
from toolkit.custom_dataset_cv import DataLoaderCIFAR10Classification
from toolkit.metrics import cv_compute_accuracy
from toolkit.utils import get_optimal_batchsize, dict_to_device
from modeling_MobileNetV1.model import MobileNetV1
from modeling_MobileNetV1.config import MobileNetV1Config

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
config = MobileNetV1Config()
config.num_classes = 10
model = MobileNetV1(config)
model.to(device)

In [4]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable Parameters: ', params)

Trainable Parameters:  3217226


In [5]:
train_loader = DataLoaderCIFAR10Classification(resize=224, train=True)
valid_loader = DataLoaderCIFAR10Classification(resize=224, train=False)

cifar10 exists...
cifar10 exists...


In [10]:
# get_optimal_batchsize(train_loader.dataset, model)

64

In [6]:
# Hyperparameters
BS = 64
EPOCHS = 3
LR = 5e-3

In [7]:
train_loader = train_loader.return_dataloader(batch_size=BS, shuffle=True)
valid_loader = valid_loader.return_dataloader(batch_size=BS, shuffle=False)
print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  782
Length of Valid Loader:  157


In [8]:
# Sanity check forward pass
model.eval()
with torch.set_grad_enabled(False):
  for sample in train_loader:
    outputs = model(**dict_to_device(sample, device))
    loss, logits = outputs[0], outputs[1]
    print(logits.shape, loss.item())
    break

torch.Size([64, 10]) 2.368525505065918


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [9]:
num_training_steps = len(train_loader) * EPOCHS
progress_bar = tqdm(range(num_training_steps))

for epoch in range(EPOCHS):
  model.train()
  for sample in train_loader:
      outputs = model(**dict_to_device(sample, device))
      loss = outputs[0]
      loss.backward()

      optimizer.step()
      optimizer.zero_grad()
      progress_bar.update(1)
  model.eval()
  with torch.set_grad_enabled(False):
    train_acc = cv_compute_accuracy(model, train_loader, device)
    valid_acc = cv_compute_accuracy(model, valid_loader, device)
  print('Train Accuracy: %.2f%% || Valid Accuracy: %.2f%%' % (train_acc, valid_acc)) 

Train Accuracy: 60.13% || Valid Accuracy: 57.36%
Train Accuracy: 67.69% || Valid Accuracy: 63.38%
Train Accuracy: 75.86% || Valid Accuracy: 69.91%
